# RAG Pipeline with LangChain

## 1. Setup and Imports

In [1]:
import os
import json
import ast
from dotenv import load_dotenv
from konlpy.tag import Okt

# LangChain components
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS # Zilliz 대신 FAISS 임포트
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever

## 2. Data Loading and Document Preparation
Load the audit cases and prepare them in LangChain's `Document` format. We create two sets of documents for our hybrid search strategy:
1.  **Full-text documents:** For semantic search (FAISS).
2.  **Keyword-focused documents:** For keyword search (BM25), using only the 'problem' and 'action' fields.

In [2]:
def load_and_prepare_docs(filepath="audit_cases.json"):
    """
    [Baseline] 'contents_summary' 필드만을 사용하여 문서를 생성합니다.
    """
    print(f"Loading data from {filepath} and creating baseline documents from summaries...")
    with open(filepath, 'r', encoding='utf-8') as f:
        audit_cases = json.load(f)

    docs = []
    for i, case in enumerate(audit_cases):
        site = case.get('site', '알 수 없음')
        category = case.get('category', '알 수 없음')
        date = case.get('date', '알 수 없음')
        original_title = case.get('title', '')
        
        metadata = {
            "index": i, "title": original_title, "site": site,
            "category": category, "date": date
        }

        summary_dict = {}
        summary_str = case.get('contents_summary')
        if summary_str:
            try:
                summary_dict = ast.literal_eval(summary_str)
            except (ValueError, SyntaxError):
                summary_dict = {}
        
        title = summary_dict.get('title_str', original_title)
        keywords = ", ".join(summary_dict.get('keyword_list', []))
        problems = summary_dict.get('problems_str', '')
        action = summary_dict.get('action_str', '')
        standards = summary_dict.get('standards_str', '')

        summary_based_text = (
            f"출처: {site}\n"
            f"분류: {category}\n"
            f"일자: {date}\n"
            f"제목: {title}\n"
            f"핵심 키워드: {keywords}\n"
            f"문제 요약: {problems}\n"
            f"조치 요약: {action}\n"
            f"관련 규정: {standards}"
        )
        docs.append(Document(page_content=summary_based_text, metadata=metadata))

    full_text_documents = docs
    keyword_documents = docs

    print(f"  - Created {len(docs)} summary-based documents for baseline.")
    return full_text_documents, keyword_documents

full_text_documents, keyword_documents = load_and_prepare_docs()

Loading data from audit_cases.json and creating baseline documents from summaries...
  - Created 4961 summary-based documents for baseline.


## 3. Retriever Setup (Hybrid Search)
We'll set up two retrievers and combine them using `EnsembleRetriever`.

In [3]:
# Initialize models and tokenizers
print("Initializing models and retrievers...")
embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")
okt = Okt()

# 1. FAISS (Semantic) Retriever
print("  - Building FAISS index...")
faiss_vectorstore = FAISS.from_documents(full_text_documents, embeddings)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 5})
print("    - FAISS retriever ready.")

# 2. BM25 (Keyword) Retriever
print("  - Building BM25 index...")
bm25_retriever = BM25Retriever.from_documents(
    documents=keyword_documents, 
    preprocess_func=lambda s: okt.morphs(s) # Use Okt for tokenization
)
bm25_retriever.k = 5
print("    - BM25 retriever ready.")

# 3. Ensemble (Hybrid) Retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[faiss_retriever, bm25_retriever],
    weights=[0.5, 0.5] # Give equal weight to semantic and keyword search
)
print("Ensemble retriever ready!")

Initializing models and retrievers...


/var/folders/14/wm3n_r1s01jfgw6n8j7pr4740000gn/T/ipykernel_10184/599536852.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")


  - Building FAISS index...
    - FAISS retriever ready.
  - Building BM25 index...
    - BM25 retriever ready.
Ensemble retriever ready!


## 4. RAG Chain Construction (LCEL)
Now we define the full RAG chain using LangChain Expression Language (LCEL).

In [4]:
prompt_template = """
당신은 감사 전문가입니다. 제공되는 '관련 감사 사례'를 근거로 하여 사용자의 '질문'에 대해 답변해 주세요. 근거가 부족하면 '정보 없음'으로
답하세요.

[관련 감사 사례]
{context}

[질문]
{question}

[답변]
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
# Gemini 대신 Ollama 모델 사용
llm = Ollama(model="gemma3:latest", base_url="http://localhost:11434", temperature=0)

# Helper function to format retrieved documents
def format_docs(docs):
    return "\n\n".join([f"### 감사사례 (제목: {doc.metadata.get('title', 'N/A')}){doc.page_content}" for doc in docs])

# RAG Chain
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
                                                                                                                                
print("RAG chain constructed successfully with Ollama (gemma3:latest).")

RAG chain constructed successfully with Ollama (gemma3:latest).


/var/folders/14/wm3n_r1s01jfgw6n8j7pr4740000gn/T/ipykernel_10184/1146039147.py:16: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:latest", base_url="http://localhost:11434", temperature=0)


## 5. Test Queries
**이 셀의 `test_query` 변수만 변경하고 이 셀만 반복적으로 실행하여 다양한 질문을 테스트할 수 있습니다.**

In [5]:
test_query = "부실시공에 따라 재시공을 하도록 한 감사건도 있나?" # 여기에 질문을 변경하세요!

print(f"--- Running RAG chain for query: '{test_query}' ---")

# Invoke the chain and stream the results
for chunk in rag_chain.stream(test_query):
    print(chunk, end="", flush=True)

print("--- Execution Complete ---")

--- Running RAG chain for query: '부실시공에 따라 재시공을 하도록 한 감사건도 있나?' ---
네, 부실시공에 따라 재시공을 하도록 한 감사건도 있습니다. 다음 감사 사례에서 확인할 수 있습니다.

*   **2023년도 부산경남본부 종합감사 결과 (한국도로공사)**: 매입부가가치세 환급업무 부적정 사례에서 “□□본부와 ◰◰지사는 휴게시설부문사업인 '◓◓(ㅍ)주유소 토양오염정밀조사 용역' 등을 매입부가 세계정으로 계리하지 않았습니다. ◰◰지사 등 3개 지사는 가로등 제어기 구매 등의 사업을 시행하면서 도로부문과 휴게시설부문을 구분하여 작성하지 않아 매입부가세계정을 누락시켰습니다. 그 결과로 부가세 15,150,045원을 과다 납부하게 되었습니다.” 이 부분은 부실시공으로 인해 발생하는 추가 비용 발생을 암시합니다.

*   **2024년도 특정감사결과_14 (국가철도공단)**: “연약 지반 통로박스 4개소의 신축이음부 이격 현상이 발생하였음. 시공 및 건설 사업 관리 업무의 부적정이 확인됨.” 이 부분은 이격 현상 발생으로 인해 재시공이 필요할 수 있음을 시사합니다.

*   **2023년 유지보수 청렴취약분야 운영실태 특정감사 결과 (한국도로공사)**: 차선도색 공사 장비 관리 부적정 사례에서 “부실시공이 발생하였음.” 이 부분은 부실시공으로 인해 재시공이 필요할 수 있음을 시사합니다.

이 외에도 다른 감사 사례에서도 부실시공으로 인해 발생하는 문제점을 간접적으로 언급하고 있습니다.--- Execution Complete ---
